# B92 Protocol

William Sanford, Blake Danziger  
Physics 75  
Lab 1  
Adapted from the implementation of the BB84 Protocol found at https://qiskit.org/textbook/ch-algorithms/quantum-key-distribution.html

## Protocol Steps
1. Alice generates a random bit string and creates a corresponding qubit string that is mapped from the bit string as follows: 0 -> |z+>, 1 -> |x+>. Alice then sends these qubits to Bob over a public channel.
2. Bob generates a random bit string and measures Alice's qubits with the following mapping from his bit string: 0 -> {|x+>, |x->}, 1 -> {|z+>, |z->}.
3. Bob sends his measurement bits back to Alice over a public channel.
4. Alice and Bob both remove all the bits in their strings corresponding to the 0s in Bob's string of measurements.

### Import Packages

In [34]:
from qiskit import QuantumCircuit, Aer, transpile, assemble
from qiskit.visualization import plot_histogram, plot_bloch_multivector
from numpy.random import randint
import numpy as np
import pandas as pd

### Define all the meta parameters we will need for the protocol
- n: the number of bits that Alice and Bob will randomly generate in the protocol
- test: boolean that will standardize the pseudo-random fucntions in the protocol and print the steps of the protocol along the way

In [35]:
n = 30
test = True
rand_seed = False

if rand_seed:
    np.random.seed(seed=0)

### Create the qubits that Alice will send across the public channel
- We define a function that takes a list of classical bits, and outputs a list of qubits that are encoded according to the following mapping: 0 -> |z+>, 1 -> |x+>

In [36]:
def alice_encode(input_arr):
    output = []
    for bit in input_arr:
        qc = QuantumCircuit(1,1)
        
        if bit == 1:
            qc.h(0)
        qc.barrier()
        output.append(qc)
    return output

In [37]:
alice_bits = randint(2, size=n)
alice_qubits = alice_encode(alice_bits)


if test:
    
    print('Alice\'s randomly generated bits:')
    print(alice_bits)
    
#     print('Alice qubits:')
#     for i in alice_qubits:
#         print(i)

Alice's randomly generated bits:
[1 0 1 1 0 1 1 1 0 1 1 0 1 0 1 0 1 1 1 1 0 0 1 0 1 0 0 0 1 1]


### Bob measures Alice's qubits based on a random bit string and sends his measurement string back to Alice across the public channel
- We define a function that takes a list of qubits and a list of classical bits. We loop through the lists in parallel, and if the corresponding classical bit is a 0, we measure the qubit in the x-basis. We measure the quibit in the z-basis otherwise. We return all of the measurement outputs in a list

In [38]:
def bob_decode(alice_qubits, bob_bits):
    output = []
    for alice_qubit, bob_bit in zip(alice_qubits, bob_bits):
        
        # Measure in the x-basis
        if bob_bit == 0:
            alice_qubit.h(0)
            alice_qubit.measure(0, 0)
        
        # Measure in the z-basis
        else:
            alice_qubit.measure(0, 0)
            
        # Simulate the measurement (taken directly from the qiskit example)
        qasm_sim = Aer.get_backend('qasm_simulator')
        qobj = assemble(alice_qubit, shots=1, memory=True)
        result = qasm_sim.run(qobj).result()
        measured_bit = int(result.get_memory()[0])
        output.append(measured_bit)
    return output

In [39]:
bob_bits = randint(2, size=n)
bob_yield = bob_decode(alice_qubits, bob_bits)

if test:
    print('Bob\'s randomly generated bits:')
    print(bob_bits)
    
    print('Bob\'s measurement yeild:')
    print(bob_yield)

Bob's randomly generated bits:
[0 0 0 0 1 0 1 1 0 1 0 0 0 0 0 0 0 1 1 0 1 1 1 0 0 0 0 1 0 0]
Bob's measurement yeild:
[0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]


### Create the shared keys
- We define a function that takes both Alice's and Bob's original bit strings, and removes all of the bits that correspond to a 0 in Bob's string of measurements. We return a tuple of stirngs that are a concatenation of these bits

In [40]:
def remove_zeros(alice_bits, bob_bits, bob_yield):
    bob_key = ''
    alice_key = ''
    
    for a_bit, b_bit, b_yield in zip(alice_bits, bob_bits, bob_yield):
        if b_yield == 1:
            bob_key += str(b_bit)
            alice_key += str(a_bit)

    return alice_key, bob_key

In [41]:
alice_key, bob_key = remove_zeros(alice_bits, bob_bits, bob_yield)

if test:
    print('Key strings')
    print(f'Alice\'s key: {alice_key}')
    print(f'Bob\'s key: {bob_key}')

Key strings
Alice's key: 01010011
Bob's key: 01010011


### Testing Results
- We can test if the protocol worked by checking if Alice and Bob have the same keys

In [42]:
if bob_key == alice_key:
    print('The protocol worked and Alice and Bob have the same keys!')
else:
    print('Something went wrong! Alice and Bob do not have the same keys!')

The protocol worked and Alice and Bob have the same keys!


### More Thorough Testing
If we want to be more thorough, we can run through the protocol many times and see if Alice's key matches Bob's key in each run. We will use the same method as above in a more condensed form.

In [43]:
def test(shots, n):
    correct = 0
    
    for i in range(shots):
        alice_bits = randint(2, size=n)
        alice_qubits = alice_encode(alice_bits)
        
        bob_bits = randint(2, size=n)
        bob_yield = bob_decode(alice_qubits, bob_bits)
        
        alice_key, bob_key = remove_zeros(alice_bits, bob_bits, bob_yield)
        
        if alice_key == bob_key:
            correct += 1
    
    print(f'After {shots} tests, Bob and Alice have matching keys {(correct/shots)*100}% of the time.')

In [44]:
test(10,30)

After 10 tests, Bob and Alice have matching keys 100.0% of the time.
